In [ ]:
!pip install streamlit pyngrok openai requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install streamlit-folium folium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 9.9 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import requests
import pandas as pd
import folium
from streamlit_folium import st_folium

st.set_page_config(page_title="Ringkasan Cuaca", layout="centered")
st.title("📍 Ringkasan Cuaca Berdasarkan Lokasi atau Kota")

API_KEY = "d4179fb703532ad460882dd59234d867"

def get_coords_from_city(city_name, api_key):
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=1&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    if data:
        return data[0]['lat'], data[0]['lon']
    else:
        return None, None

def get_forecast_summary(lat, lon, api_key):
    url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()

    if response.status_code != 200:
        return None, "Gagal mengambil data cuaca."

    forecasts = data["list"]
    rainfalls = []
    temps = []
    humidities = []

    for entry in forecasts:
        temps.append(entry['main']['temp'])
        humidities.append(entry['main']['humidity'])
        rain = entry.get('rain', {}).get('3h', 0)
        rainfalls.append(rain)

    summary = {
        "Rata-rata Suhu (°C)": round(sum(temps) / len(temps), 2),
        "Rata-rata Kelembapan (%)": round(sum(humidities) / len(humidities), 2),
        "Rata-rata Curah Hujan (mm/3 jam)": round(sum(rainfalls) / len(rainfalls), 2)
    }

    return summary, None

# --- Input nama kota ---
st.subheader("🔤 Masukkan Nama Kota (Opsional)")
city = st.text_input("Contoh: Bandung")

if city:
    lat, lon = get_coords_from_city(city, API_KEY)
    if lat is not None:
        st.success(f"Koordinat ditemukan untuk {city}: ({lat:.2f}, {lon:.2f})")
        summary_data, error = get_forecast_summary(lat, lon, API_KEY)
        if summary_data:
            st.subheader("📊 Ringkasan Cuaca dari Nama Kota")
            for key, value in summary_data.items():
                st.write(f"✅ **{key}:** {value}")
        else:
            st.error(error)
    else:
        st.warning("Kota tidak ditemukan. Anda bisa memilih lokasi lewat peta di bawah.")

# --- Peta interaktif ---
st.divider()
st.subheader("🗺️ Atau Klik Lokasi di Peta")

m = folium.Map(location=[-2.5, 118], zoom_start=5)
folium.LatLngPopup().add_to(m)
map_data = st_folium(m, width=700, height=500)

if map_data and map_data.get("last_clicked"):
    lat = map_data["last_clicked"]["lat"]
    lon = map_data["last_clicked"]["lng"]
    st.success(f"Lokasi dipilih: Latitude={lat:.2f}, Longitude={lon:.2f}")

    summary_data, error = get_forecast_summary(lat, lon, API_KEY)
    if summary_data:
        st.subheader("📊 Ringkasan Cuaca dari Lokasi Peta")
        for key, value in summary_data.items():
            st.write(f"✅ **{key}:** {value}")
    else:
        st.error(error)
else:
    st.info("Klik lokasi di peta untuk melihat data cuaca.")


Writing app.py


In [ ]:
from pyngrok import conf, ngrok

# Masukkan token ngrok kamu di sini
ngrok.set_auth_token("2xH5SFTP0InDNxn9Xe9IHmD49q8_3avvKtEwLivdV5NvZJzfQ")


In [ ]:
!streamlit run app.py &>/content/log.txt &

# Hubungkan ke port 8501
public_url = ngrok.connect(addr="http://localhost:8501")
print("Akses aplikasi Anda di:", public_url)


Akses aplikasi Anda di: NgrokTunnel: "https://c183-34-90-52-240.ngrok-free.app" -> "http://localhost:8501"
